In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit_metal import MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

In [3]:
%metal_heading Making your design in Metal

In [4]:
design = designs.MultiPlanar({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [6]:
# Making the qubit
q_opts = dict(
    pos_x = "2mm",
    pos_y = "2mm",
    connection_pads = dict(
        readout = dict()
    )
)

q1 = TransmonPocket(design, "Q1", options=q_opts)

gui.rebuild()
gui.autoscale()

In [7]:
# Making the open to ground
otg_opts = dict(
    pos_x = "4mm",
    pos_y = "2mm"
)

otg = OpenToGround(design, "otg", options=otg_opts)

gui.rebuild()
gui.autoscale()

In [8]:
meander_opts = dict(
    pin_inputs = dict(
        start_pin = dict(component = "Q1", pin = "readout"),
        end_pin = dict(component = "otg", pin = "open")
    ),
    lead = dict(
        start_straight = "100um"
    ),
    fillet = "99.99um",
    asymmetry = "100um"
)

mtl = RouteMeander(design, "mtl", options=meander_opts)

gui.rebuild()
gui.autoscale()

In [9]:
%metal_heading Render to Gmsh

In [15]:
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer

# Instantiate QGmshRenderer
gmsh_renderer = QGmshRenderer(design)

# Set necessary options for mesh generation
gmsh_renderer.options.mesh.min_size = "5um"
gmsh_renderer.options.mesh.max_size = "70um"

In [16]:
# Render the wireframe of the design
gmsh_renderer.render_design(mesh_geoms=False, skip_junctions=True)

# Launch Gmsh GUI to verify
gmsh_renderer.launch_gui()

Info    : Clearing all models and views...
Info    : Done clearing all models and views
-------------------------------------------------------
Version       : 4.10.3
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20220526
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [17]:
# Once you've verified that the design looks correct
gmsh_renderer.add_mesh(dim=3, intelli_mesh=False)
# gmsh_renderer.launch_gui()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 10 (Line)
Info    : [  0%] Meshing curve 1 (Line)
Info    : [  0%] Meshing curve 9 (Line)
Info    : [  0%] Meshing curve 409 (Line)
Info    : [  0%] Meshing curve 411 (Line)
Info    : [  0%] Meshing curve 410 (Line)
Info    : [  0%] Meshing curve 412 (Line)
Info    : [ 10%] Meshing curve 413 (Line)
Info    : [ 10%] Meshing curve 414 (Line)
Info    : [ 10%] Meshing curve 415 (Line)
Info    : [ 10%] Meshing curve 416 (Line)
Info    : [ 10%] Meshing curve 417 (Line)
Info    : [ 10%] Meshing curve 418 (Line)
Info    : [ 10%] Meshing curve 420 (Line)
Info    : [ 10%] Meshing curve 419 (Line)
Info    : [ 10%] Meshing curve 421 (Line)
Info    : [ 10%] Meshing curve 422 (Line)
Info    : [ 10%] Meshing curve 423 (Line)
Info    : [ 10%] Meshing curve 424 (Line)
Info    : [ 10%] Meshing curve 425 (Line)
Info    : [ 10%] Meshing curve 426 (Line)
Info    : [ 10%] Meshing curve 427 (Line)
Info    : [ 10%] Meshing curve 428 (Line)
Info    : [ 10%

In [22]:
gmsh_renderer.export_mesh("./mesh/test.msh")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/abeerv/Documents/coding/qiskit-metal/metal_os/lib64/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_36983/1021809057.py", line 1, in <cell line: 1>
    gmsh_renderer.export_mesh("./mesh/test.msh")
  File "/home/abeerv/Documents/coding/qiskit-metal/qiskit_metal/renderers/renderer_gmsh/gmsh_renderer.py", line 1073, in export_mesh
    raise ValueError(f"Directory not found: {par_dir}")
ValueError: Directory not found: /home/abeerv/Documents/coding/qiskit-metal/tutorials/3 Renderers/mesh

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/abeerv/Documents/coding/qiskit-metal/metal_os/lib64/python3.9/site-packages/IPython/core/interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/abeerv/Documents/coding/qisk

In [23]:
# Close and destroy the renderer object
gmsh_renderer.close()

Info    : Clearing all models and views...
Info    : Done clearing all models and views


True

In [5]:
%metal_heading Intelli-mesh in Gmsh

In [13]:
gmsh_renderer = QGmshRenderer(design)
gmsh_renderer.options.mesh.min_size = "5um"
gmsh_renderer.options.mesh.max_size = "70um"

gmsh_renderer.render_design(mesh_geoms=False)
gmsh_renderer.add_mesh(dim=3, intelli_mesh=True)

gmsh_renderer.launch_gui()

# Close and destroy the renderer object
gmsh_renderer.close()

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [  0%] Meshing curve 10 (Line)
Info    : [  0%] Meshing curve 11 (Line)
Info    : [  0%] Meshing curve 410 (Line)
Info    : [  0%] Meshing curve 409 (Line)
Info    : [  0%] Meshing curve 411 (Line)
Info    : [  0%] Meshing curve 9 (Line)
Info    : [  0%] Meshing curve 412 (Line)
Info    : [ 10%] Meshing curve 413 (Line)
Info    : [ 10%] Meshing curve 414 (Line)
Info    : [ 10%] Meshing curve 415 (Line)
Info    : [ 10%] Meshing curve 416 (Line)
Info    : [ 10%] Meshing curve 417 (Line)
Info    : [ 10%] Meshing curve 418 (Line)
Info    : [ 10%] Meshing curve 419 (Line)
Info    : [ 10%] Meshing curve 420 (Line)
Info    : [ 10%] Meshing curve 421 (Line)
Info    : [ 10%] Meshing curve 422 (Line)
Info    : [ 10%] Meshing curve 423 (Line)
Info    : [ 10%] Meshing curve 424 (Line)
Info    : [ 10%] Meshing curve 425 (Line)
Info    : [ 

True

In [14]:
gui.main_window.close()

True